In [ ]:
%pip install openpyxl pandas


In [ ]:
start_date = '2025-04-01'
end_date = '2025-07-01'

In [ ]:
# get most recent file starting with 'RC101533' in '/home/martin/Downloads'
import os
import glob 

downloads_path = '/home/martin/Downloads'
file_pattern = os.path.join(downloads_path, 'RC101533*PODLIST.xlsx')
files = glob.glob(file_pattern)
if files:
    most_recent_file = max(files, key=os.path.getctime)
else:
    most_recent_file = None
print(f"Most recent file: {most_recent_file}")

In [ ]:

#read the file to dataframe
import pandas as pd
df = pd.read_excel(most_recent_file)

# convert 'aktiv seit' and 'aktiv bis' to format comparable with start_date and end_date
df['aktiv seit'] = pd.to_datetime(df['aktiv seit'], errors='coerce').dt.strftime('%Y-%m-%d')
df['aktiviert'] = pd.to_datetime(df['aktiviert'], errors='coerce').dt.strftime('%Y-%m-%d')

#df['aktiv bis'] = pd.to_datetime(df['aktiv bis'], errors='coerce').dt.strftime('%Y-%m-%d')

# if 'aktiv bis' is 9999-12-31, set it to NaN
df['aktiv bis'] = df['aktiv bis'].replace('9999-12-31', pd.NaT)
df['aktiv bis'] = pd.to_datetime(df['aktiv bis'], errors='coerce').dt.strftime('%Y-%m-%d')

In [ ]:
df

In [ ]:
# filter by 'aktiv seit' smaller than end_date and 'aktiv bis' larger than start_date or NaN
df_filtered = df[(df['aktiv seit'] < end_date) & (df['aktiv bis'] >= start_date) | df['aktiv bis'].isna()]

In [ ]:
df_filtered

In [ ]:
# get most recent file 'RC101533-EEG-Masterdata*.xlsx' in '/home/martin/Downloads'
file_pattern = os.path.join(downloads_path, 'RC101533-EEG-Masterdata*.xlsx')
files = glob.glob(file_pattern)
if files:
    most_recent_file_mitglieder = max(files, key=os.path.getctime)
else:
    most_recent_file_mitglieder = None
print(f"Most recent file mitglieder: {most_recent_file_mitglieder}")    

In [ ]:
df_mitglieder = pd.read_excel(most_recent_file_mitglieder, sheet_name='Mitglieder')

# convert 'aktiviert' and 'deaktiviert' to format comparable with start_date and end_date
df_mitglieder['aktiviert'] = pd.to_datetime(df_mitglieder['aktiviert'], errors='coerce').dt.strftime('%Y-%m-%d')
df_mitglieder['deaktiviert'] = pd.to_datetime(df_mitglieder['deaktiviert'], errors='coerce').dt.strftime('%Y-%m-%d')


In [ ]:
# select only columns 'Zählpunkt', 'ZP-Status', 'aktiviert', 'Mitgliedstarif', 'deaktiviert'
#df_mitglieder = df_mitglieder[['Zählpunkt', 'ZP-Status', 'aktiviert', 'Mitgliedstarif', 'deaktiviert']]

In [ ]:
# only unique rows
df_mitglieder = df_mitglieder.drop_duplicates()


In [ ]:
# filter df_mitglieder by 'aktiviert' < 2025-07-01 and 'deaktiviert' >= 2025-04-01 or NaN
# and ZP-Status is 'ACTIVE'
df_mitglieder_filtered = df_mitglieder[(df_mitglieder['aktiviert'] < end_date) & (df_mitglieder['deaktiviert'] >= start_date) | df_mitglieder['deaktiviert'].isna() & (df_mitglieder['ZP-Status'] == 'ACTIVE')] 


In [ ]:
df_mitglieder_filtered['Zp. Tarifname'].unique()

# Data checks

In [ ]:
# iterate df_mitglieder_filtered

for index, row in df_mitglieder_filtered.iterrows():
    zählpunkt = row['Zählpunkt']
    status = row['ZP-Status']
    aktiviert = row['aktiviert']
    tarif = row['Mitgliedstarif']
    deaktiviert = row['deaktiviert']
    #print(f"Zählpunkt: {zählpunkt}, Status: {status}, Aktiviert: {aktiviert}, Tarif: {tarif}, Deaktiviert: {deaktiviert}")
    # check if zählpunkt is in df_filtered
    if zählpunkt not in df_filtered['Zählpunktname'].values:
        print(f"Zählpunkt {zählpunkt} is NOT in df_filtered")
        print(f"Status: {status}, Aktiviert: {aktiviert}, Tarif: {tarif}, Deaktiviert: {deaktiviert}")
    else:    
        # get row from df_filtered
        row = df_filtered[df_filtered['Zählpunktname'] == zählpunkt]
        netz_aktiviert = row['aktiv seit'].values[0]

        

        # TODO: branch not working
        if netz_aktiviert != aktiviert:
            print("falsches datum")
            print(zählpunkt)
            print(aktiviert)
            print(netz_aktiviert)
            print("\n")

    
    


In [ ]:
# iterate df_filtered
for index, row in df_filtered.iterrows():
    zählpunkt = row['Zählpunktname']
    aktiv_seit = row['aktiv seit']
    aktiv_bis = row['aktiv bis']
    # print(f"Zählpunkt: {zählpunkt}, Aktiv seit: {aktiv_seit}")
    # check if zählpunkt is in df_mitglieder_filtered
    if zählpunkt not in df_mitglieder_filtered['Zählpunkt'].values:
        print(f"Zählpunkt {zählpunkt} is NOT in df_mitglieder_filtered")
        print(f"Aktiv seit: {aktiv_seit} Aktiv bis: {aktiv_bis}")

In [ ]:
# filter all rows of df_mitglieder_filtered where Mitgliedstarif is empty
df_mitglieder_filtered_empty_tarif = df_mitglieder_filtered[df_mitglieder_filtered['Mitgliedstarif'].isnull()]
df_mitglieder_filtered_empty_tarif

In [ ]:
# alle start daten checken

# vergleichsdaten summe Q2
# gesamte erzeugung: 213228.51
# eigendeckung: 60689.45
# restüberschuss: 152539.06

